In [1]:
# Dependencies
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# File paths and Tensorflow
import os
import tensorflow as tf

# Sklearn scaling
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

# Keras
from tensorflow import keras
# from tensorflow.keras

In [14]:
# Read in the liver_clean.csv
liver_clean_df = pd.read_csv("/liver_boolean.csv")
liver_clean_df

,Unnamed: 0,N_Days,Status,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,...,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Stage_1,Stage_2,Stage_3
0,0,2221,C,Placebo,18499,F,N,Y,N,N,...,227.0,598.0,52.70,57.000000,256.0,9.9,1,True,False,False
1,1,1230,C,Placebo,19724,M,Y,N,Y,N,...,22.0,663.0,45.00,75.000000,220.0,10.8,2,False,True,False
2,2,4184,C,Placebo,11839,F,N,N,N,N,...,51.0,1243.0,122.45,80.000000,225.0,10.0,2,False,True,False
3,3,2090,D,Placebo,16467,F,N,N,N,N,...,23.0,1024.0,77.50,58.000000,151.0,10.2,2,False,True,False
4,4,2105,D,Placebo,21699,F,N,Y,N,N,...,74.0,1052.0,108.50,109.000000,151.0,11.5,1,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,24995,3584,D,D-penicillamine,23612,F,N,N,N,N,...,173.0,9009.8,127.71,96.000000,295.0,11.0,2,False,True,False
24996,24996,3584,D,D-penicillamine,23612,F,N,N,N,N,...,173.0,9009.8,127.71,96.000000,295.0,11.0,2,False,True,False
24997,24997,971,D,D-penicillamine,16736,F,N,Y,Y,Y,...,18.0,790.0,179.80,124.702128,104.0,13.0,3,False,False,True
24998,24998,3707,C,D-penicillamine,16990,F,N,Y,N,N,...,13.0,1637.0,170.50,70.000000,426.0,10.9,2,False,True,False
